## প্রয়োজনীয় লাইব্রেরি আমদানি করুন
Phi-4 মডেল লোড এবং ব্যবহার করার জন্য প্রয়োজনীয় PyTorch এবং transformers লাইব্রেরি আমদানি করুন।


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## র্যান্ডম সিড সেট করুন
র্যান্ডম সিড সেট করুন যাতে বিভিন্ন রান জুড়ে পুনরুত্পাদনযোগ্য ফলাফল নিশ্চিত করা যায়।


In [ ]:
torch.random.manual_seed(0)

## ফি-৪-মিনি-ফ্ল্যাশ-রিজনিং মডেল এবং টোকেনাইজার লোড করুন
মাইক্রোসফট ফি-৪-মিনি-ফ্ল্যাশ-রিজনিং মডেল এবং এর সংশ্লিষ্ট টোকেনাইজার হাগিং ফেস থেকে লোড করুন। মডেলটি দ্রুত ইনফারেন্সের জন্য CUDA-তে লোড করা হবে।


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## ইনপুট বার্তা প্রস্তুত করুন
একটি কথোপকথনের বার্তা তৈরি করুন যেখানে একটি দ্বিঘাত সমীকরণের গণিত সমস্যা থাকবে এবং এটি মডেলের জন্য চ্যাট টেমপ্লেট ব্যবহার করে ফরম্যাট করুন।


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## প্রতিক্রিয়া তৈরি করুন
নির্ধারিত প্যারামিটার যেমন temperature এবং top_p ব্যবহার করে মডেল থেকে একটি প্রতিক্রিয়া তৈরি করুন, যাতে আউটপুটে নিয়ন্ত্রিত র্যান্ডমনেস নিশ্চিত করা যায়।


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## আউটপুটকে টেক্সটে রূপান্তর করুন
জেনারেট করা টোকেন সিকোয়েন্সগুলোকে মানুষের পড়ার উপযোগী টেক্সটে রূপান্তর করুন, যেখানে মূল ইনপুট টোকেনগুলো বাদ দিয়ে শুধুমাত্র মডেলের প্রতিক্রিয়া প্রদর্শিত হবে।


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
